In [1]:
# imports
import sys
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np
import mdtraj

In [3]:
# load files
psf = app.CharmmPsfFile('init.psf')
pdbfile = app.PDBFile('init.pdb')
params = app.CharmmParameterSet('toppar/top_all36_prot.rtf', 'toppar/par_all36m_prot.prm', 'toppar/toppar_water_ions.str')
positions = pdbfile.getPositions(asNumpy=True)

In [4]:
# system
psf.setBox(39.0*unit.angstroms, 39.0*unit.angstroms, 39.0*unit.angstroms)

system = psf.createSystem(params,
    nonbondedMethod=app.PME, 
    nonbondedCutoff=1.2*unit.nanometers,
    implicitSolvent=None,
    constraints=app.HBonds,
    rigidWater=True, 
    verbose=True, 
    ewaldErrorTolerance=0.0005)

barostat = openmm.MonteCarloBarostat(1.0*unit.bar, 300.0*unit.kelvin)
system.addForce(barostat)

Adding particles...
Adding lonepairs...
Adding bonds...
Adding angles...
Adding Urey-Bradley terms
Adding torsions...
Adding impropers...
Adding CMAP coupled torsions...
Adding nonbonded interactions...


8

In [5]:
# integrator
integrator = openmm.LangevinIntegrator(300.0*unit.kelvin,   # Temperature of head bath
                                       1.0/unit.picosecond, # Friction coefficient
                                       0.002*unit.picoseconds) # Time step

In [6]:
# platform
platform = openmm.Platform.getPlatformByName('CPU')
# platform = openmm.Platform.getPlatformByName('CPU')
# prop = dict(CudaPrecision='mixed')

In [7]:
# Build simulation context
simulation = app.Simulation(psf.topology, system, integrator, platform)
#simulation = app.Simulation(psf.topology, system, integrator, platform, prop)
simulation.context.setPositions(positions)

In [8]:
# load checkpoint
with open('run.chk', 'rb') as f:
    simulation.context.loadCheckpoint(f.read())

In [9]:
# energy after checkpoint
print("\nafter checkpoint")
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

after restart
-80328.48698999993 kJ/mol


In [10]:
# simulation
reportInterval = 10
nstep = 1000
simulation.reporters.append(app.StateDataReporter(sys.stdout, reportInterval, step=True, time=True, potentialEnergy=True, temperature=True, progress=True, remainingTime=True, speed=True, totalSteps=nstep, separator='\t'))
simulation.reporters.append(mdtraj.reporters.HDF5Reporter('run2.h5', reportInterval, coordinates=True, time=True, cell=True, potentialEnergy=True, temperature=True))
simulation.reporters.append(mdtraj.reporters.DCDReporter('run2.dcd', reportInterval))
simulation.step(nstep)

#"Progress (%)"	"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
1.0%	10	2.019999999999999	-79935.48502287328	251.70821069456468	0	--


/opt/conda/lib/python3.7/site-packages/mdtraj/utils/validation.py:116: TypeCastPerformanceWarning: Casting xyz dtype=float64 to <class 'numpy.float32'> 
  TypeCastPerformanceWarning)


2.0%	20	2.039999999999997	-79857.75614055035	251.40329371969088	9.86	0:17
3.0%	30	2.0599999999999947	-80169.24859748456	257.0160341634953	7.08	0:23
4.0%	40	2.0799999999999925	-80176.00611961771	259.46457068563376	7.14	0:23
5.0%	50	2.0999999999999903	-79775.33862739924	253.17829762995197	6.47	0:25
6.0%	60	2.119999999999988	-79957.4772374103	260.45715702939646	6.56	0:24
7.0%	70	2.139999999999986	-79920.2956901644	259.6700887640571	6.61	0:24
8.0%	80	2.1599999999999837	-79705.59813714374	257.021872228278	6.27	0:25
9.0%	90	2.1799999999999815	-79764.09176156604	260.7464659862931	6.35	0:24
10.0%	100	2.1999999999999793	-79466.45964449315	256.113408969286	6.23	0:24
11.0%	110	2.219999999999977	-79592.66655287132	259.565220456151	6.37	0:24
12.0%	120	2.239999999999975	-79275.60716198206	252.17229409858479	6.45	0:23
13.0%	130	2.2599999999999727	-79697.80673879333	261.2584975294003	6.33	0:23
14.0%	140	2.2799999999999705	-79226.33971465238	251.65583818555942	6.4	0:23
15.0%	150	2.2999999999999683	-794

In [11]:
with open('run2.chk', 'wb') as f:
    f.write(simulation.context.createCheckpoint())

In [12]:
del simulation # Make sure to close all files